**Imports**

PyTorch Geometric

In [1]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 12.2 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 6.6 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.5 MB 11.3 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 750 kB 11.3 MB/s 
     |████████████████████████████████| 407 kB 12.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=59519c3fa1318c4e97e627aa78f3cd2f99a11d7cc174f26e78243a17487f5cf6
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import pandas as pd
import numpy as np

from torch_geometric.data import InMemoryDataset, download_url
from datetime import datetime

**Mount GDrive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
action_types.csv              participants_activity_anomalies.csv
activities.csv                patients.csv
activity_types.csv            scores.csv
anomalies.csv                 scores_predictions.csv
anomaly_types.csv             sensor_locations.csv
diagnosis_predictions.csv     sensors.csv
diagnosis_types.csv           tasks.csv
events.csv                    task_types.csv
overt_error_action_types.csv  trajectory_anomalies.csv


In [65]:
def task_activity_sensors(tasks, events, sensors, activities, patient):

  # Seleziona i task-activity eseguiti dal paziente 'patient'
  task_activity = tasks.query('patient == ' + patient)

  # Seleziona il tempo di inizio e fine delle attività svolte dal paziente 'patient'
  activities_patient = activities.query('patient == ' + patient)
  start_time = activities_patient.iloc[0]['start']
  end_time = activities_patient.iloc[len(activities_patient) - 1]['end']

  # Query nel dataframe events per selezionare solo i sensori utilizzati nelle attività
  sensors_events = events.query('patient == ' + patient + ' and time >= "' + start_time + '" and time <= "' + end_time + '"')

  # Matrice risultato che contiene i dati dei nodi
  res_node = pd.DataFrame(columns=['features'])
  res_edge = pd.DataFrame(columns=['times'])

  # Dimensione dell'array delle features
  dim_features = len(sensors)

  # Dizionario che contiene la posizione del sensore nell'array delle features
  dict_sens = {}

  for i in range(0, dim_features):
    dict_sens[sensors.iloc[i]['sensor_id']] = i

  # Indice del task-activity
  i = 0

  # Nome del task-activity precedente
  pair_str_prev = ""

  # Flag per salvataggio features di nodo e arco
  ins_upd = False
  last_node = False

  while i < len(task_activity):

    # Controlla se il task-activity attuale è l'ultimo
    if (i + 1) >= len(task_activity):
      last_node = True

    # Nome del task-activity attuale
    pair_str = str(task_activity.iloc[i]['task']) + '-' + str(task_activity.iloc[i]['activity'])

    # Controlla se il task-activity attuale e quello precedente sono diversi
    if pair_str != pair_str_prev:

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:
        # Viene salvato lo start time
        s_time = task_activity.iloc[i]['time']

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str
          i = i + 1

        else:
          ins_upd = False
          pair_str_prev = pair_str
          i = i + 1
      
      else:
        ins_upd = False
        i = i + 1
    
    else:
      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str
          i = i + 1
        
        else:
          ins_upd = False
          pair_str_prev = pair_str
          i = i + 1
      
      else:
        # Viene salvato l'end time
        e_time = task_activity.iloc[i]['time']

        ins_upd = True
        pair_str_prev = pair_str
        i = i + 1

    # Controlla se start ed end time sono stati trovati
    if ins_upd:

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:
        # Controlla se esiste un arco per il task-activity attuale 
        if pair_str in res_edge.index:

          # Aggiunge start ed end time all'arco
          #res_edge.at[pair_str, 'times'] = res_edge.loc[pair_str, 'times'].append((s_time, e_time))
          res_edge.loc[pair_str, 'times'] += [(s_time, e_time)]
        
        else:
          # Crea un nuovo arco
          res_edge.loc[pair_str] = [[(s_time, e_time)]]

      # Controlla se non esiste un nodo per il task-activity attuale
      if not pair_str in res_node.index:

        # Crea un array vuoto per le fueatures del nodo
        features_node = np.zeros(dim_features, dtype=np.int8)

        # Seleziona i sensori utilizzati per il task-activity attuale
        task_activity_sensors = sensors_events.query('time >= "' + s_time + '" and time <= "' + e_time + '"')

        # Popola l'array delle features
        for _, s in task_activity_sensors.iterrows():
          features_node[dict_sens[s['sensor']]] = 1

        # Salva nel nodo l'array delle features
        res_node.loc[pair_str] = [features_node]
  
  return res_edge, res_node

In [66]:
sensors = pd.read_csv('drive/MyDrive/CASAS400/sensors.csv')
events = pd.read_csv('drive/MyDrive/CASAS400/events.csv')
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
activities = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

edge, node = task_activity_sensors(tasks, events, sensors, activities, '88')

In [4]:
pd.read_csv('drive/MyDrive/CASAS400/sensors.csv')

,sensor_id,description
0,d001,door
1,d002,door
2,d003,door
3,d004,door
4,d005,door
...,...,...
69,m048,motion
70,m049,motion
71,m050,motion
72,m051,motion


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/events.csv')

,patient,time,sensor,value
0,88,09:00:05.78693,m010,on
1,88,09:00:08.08177,m010,off
2,88,09:00:13.71834,m013,off
3,88,09:00:19.76291,m010,on
4,88,09:00:21.6229,m013,on
...,...,...,...,...
1209934,329,16:20:29.92485,m022,off
1209935,329,16:20:32.888,m021,off
1209936,329,16:20:32.91215,m025,off
1209937,329,16:20:33.9529,d001,close


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

,patient,activity_type,start,end
0,88,1,09:21:17.2387,09:24:10.78761
1,88,2,09:25:15.5674,09:28:42.21513
2,88,3,09:30:21.23501,09:33:50.33897
3,88,4,09:34:53.77164,09:42:28.82678
4,88,5,09:43:36.54254,09:44:33.83539
...,...,...,...,...
5274,329,11,14:36:47.11877,14:40:02.51782
5275,329,12,14:44:22.30159,14:47:07.77214
5276,329,13,14:48:19.93743,14:52:49.75592
5277,329,14,14:53:32.62822,15:02:49.75547


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/task_types.csv')

,activity_id,task_id,description,action_type
0,1,1,Participant retrieves broom from supply closet,1
1,1,2,Participant retrieves duster from supply closet,1
2,1,3,Participant retrieves dust pan and brush from ...,1
3,1,4,Participant sweeps kitchen floor,5
4,1,5,Participant uses dust pan and brush,5
...,...,...,...,...
104,16,3,Participant brings deck of cards to the dining...,5
105,16,4,Participant sets up card game,5
106,16,5,Participant answers the telephone,11
107,16,6,Participant explains the rules of the game ove...,5


In [ ]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to `self.raw_dir`.
        download_url(url, self.raw_dir)
        ...

    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])